# VI. Types, type hierarchy, and multiple dispatch

In Julia types play a central role. We've covered enough so far that you can write Julia code as you would in other languages, but having an understanding of types will give you a better understanding of how Julia works and hopefully help you write better Julia code. 

In Julia, everything has a type. As we've already seen, you can figure out an object's type using the __typeof__ funciton.

In [ ]:
typeof(1.3)

In [ ]:
typeof(22)

In [ ]:
typeof("julia")

In [ ]:
typeof(3//2)

In [ ]:
typeof(3 + 2im)

In [ ]:
typeof(randn(5,5))

The curly braces indicate the type is a **parametric** type. So *Array* is a parametric type since it's a type with type parameters (in this case *Float64* and *2*).

You can use the __eltype__ function to determine the element type.

In [ ]:
A=rand(5,5)

In [ ]:
eltype(A)

In [ ]:
t = ("summer", 32, 4.3,)

In [ ]:
typeof(t)

In [ ]:
eltype(t)

Because of Julia's emphasis on types you can sometimes run into unexpected problems. Let's generate a random array of integers:

In [ ]:
A = rand(1:20, 5, 5)

As stated before, arrays are mutable:

In [ ]:
A[1,1] = 19

In [ ]:
A

Let's now set element A[1,2] equal to 3.2:

In [ ]:
A[1,2] = 3.2

What happened here?

### How types are organized in Julia:

In Julia, types are organized according according to a tree like  structure. At the top of the tree is the *Any* type. So all types are subtypes of *Any*. At the bottom of the tree (the leaves of the tree) are __concrete types__, i.e. Float64, Int64, UInt32, etc. A type can have at most one parent, but possibly more than one child.

To get a sense of how the type hierarchy is organized you can use the functions __subtypes__ and __supertype__. Let's start with a simple concrete type *Float64* and traverse the tree upwards.

In [ ]:
supertype(Float64)

So the parent of *Float64* is a type called *AbstractFloat* which in Julia is called an **abstract type**.

In [ ]:
supertype(AbstractFloat)

In [ ]:
supertype(Real)

In [ ]:
supertype(Number)

In [ ]:
supertype(Any)

From the above ouptut, we see that *Real*, *Number*, etc. are also **abstract types** (i.e. they can have children). When you create variables in Julia they can only be __concrete types__, i.e. you can not instantiate a variable of type *Real*.

Let's now start with the *Real* type and use the **subtypes** function to go down the tree.

In [ ]:
subtypes(Real)

So *Real* has four subtypes: *AbstractFloat*, *Integer*, *Irrational*, *Rational*.

In [ ]:
subtypes(Integer)

In [ ]:
subtypes(Signed)

In [ ]:
subtypes(Int64)

We can see that *Int64* is a **concrete type**. There are also suptype and supertype operators you can use to ask questions if a type is a supertype or subtype of some other type. Here we test if *Real* is a subtype of *Number*:

In [ ]:
Real <: Number

Is *Int64* a subtype of *Float64*?

In [ ]:
Int64 <: Float64

Let's test if *Real* is a supertype of *Rational*:

In [ ]:
Real >: Rational

Is *Int64* a supertype of *Complex*?

In [ ]:
Int64 >: Complex

Types are important in Julia for many reasons, but perhaps most obviously due to Julia's approach to calling functions known as **multiple dispatch**.

In Julia many versions of the same named function can exist and these different versions are called **methods**. The methods differ in that they have different function signatures. In general, these different methods correspond to a specific type of behavior by the function determined by the type of inputs.

You can see this with the built-in multiplication function __*__:

In [ ]:
*(3,2.2)

In [ ]:
*("string","input")

As we can see the behavior of __*__ is different depending on the input types: for numbers it does multiplication and for strings it does concatenation. You can see all the methods for a function using **methods**:

In [ ]:
methods(*)

What Julia does at runtime is call the specialized version of the function (i.e. method) that corresponds to the type of input arguments being passed.

You can incorporate this type of behavior into your own functions. Let's write a function that takes a Float64 squares it, and returns the result.

In [ ]:
function MySquareSpecific(x::Float64)
    return x^2
end

Passing a *Float64* should work, but passing an *Int64* will not:

In [ ]:
MySquareSpecific(3.2)

In [ ]:
MySquareSpecific(2)

To remedy this we could write another method for MySquuareSpecific to work on an Int64.

In [ ]:
function MySquareSpecific(x::Int64)
    return x^2
end

Our function now has two methods:

In [ ]:
methods(MySquareSpecific)

In [ ]:
round(MySquareSpecific(3.2), digits = 4)

In [ ]:
MySquareSpecific(2)

Though the function will not work on arguments of the Rational type which is something we might want.

In [ ]:
MySquareSpecific(3//2)

Instead of writing multiple versions of our function we could've written a generalized version to work on anything that's a subtype of *Real*.

In [ ]:
function MySquareGen(x :: Real)
    return x.^2
end

In [ ]:
a,b,c = MySquareGen(3.2), MySquareGen(2), MySquareGen(3//2)

However, in our case the best thing to do would be to leave the input type unspecified and simply do:

In [ ]:
function MySquare(x)
    return x^2
end

Reason being, Julia will do type inference on the argument being passed in. The first time it encounters this argument type it will infer the argument type, then compile and cache this compiled version of the function for the inferred argument type. The next time the function is called with the same argument type Julia will call the specialized compiled version that it has stored in memory.

This is to say that you genereally don't need or even want to create methods for every conceivable argument type; specifically in cases where Julia can infer the type and do the correct behavior.

For your reference, if you wanted to define a function whose input was an Array of Float64s that would look something like the following:

In [ ]:
function ArrFloat(A::Array{Float64})
    #statements
end

And if you wanted to define a function whose input was an Array whose elements are restricted to being subtypes of *Real*:

In [ ]:
function ArrReal(A::Array{Real})
     return 1 + A
end

# Exercise 6
* Find the supertype of the Bool type.
* Is Int64 subtype of Bool?
* Display all the methods for the "isless" function.

In this lesson we covered:
* Julia types
* Type operations
* Mulitple dispatch